In [9]:
# from process_multi_task_dataset import *
from utils.multitask_data import CustomMultiTaskDataset

In [12]:
from IPython.display import display
import pandas as pd


In [10]:
val_dataset = CustomMultiTaskDataset(split='val', task='fold', gearnet=True, alpha_only=False, root_dir = './datasets/MultiTask_fold')

Initializing MultiTask Dataset...
Start loading cached Multitask files...
Complexes Before Checking: 6
Checking the dataset...
Cutting Threshold of atom numbers: 24306


100%|██████████| 6/6 [00:00<00:00, 614.19it/s]

Complexes Before Task Selection: 6
Using Fold val Dataset for training:
now
new_complexes 6
Dataset size: 6


In [11]:
val_dataset.processed_complexes[5]

{'complex_id': '1de4',
 'num_proteins': 3,
 'labels': {'uniprots': ['P61769', 'Q30201', 'P02786'],
  'ec': [-1, -1, -1],
  'go': [-1, -1, -1],
  'reaction': [-1, -1, -1],
  'domains': ['d1de4c1', 'd1de4c2', 'd1de4a1', 'd1de4a2'],
  'fold': [[46], [465], [284], [623]],
  'ppi': 7.62,
  'lba': -1},
 'atoms_protein':       chain resname element name          x          y           z domain_name
 0         A     ARG       N    N  59.716999  59.879002  247.330002       other
 1         A     ARG       C   CA  60.457001  61.174999  247.408997       other
 2         A     ARG       C    C  61.854000  61.002998  248.013000       other
 3         A     ARG       O    O  62.028000  60.966999  249.235001       other
 4         A     ARG       C   CB  59.650002  62.205002  248.222000       other
 ...     ...     ...     ...  ...        ...        ...         ...         ...
 24301     I     NAG       O   O4  51.841000  72.031998  133.104004       other
 24302     I     NAG       O   O5  49.580002 

In [14]:
display(val_dataset.processed_complexes[5]['atoms_protein'])

,chain,resname,element,name,x,y,z,domain_name
0,A,ARG,N,N,59.716999,59.879002,247.330002,other
1,A,ARG,C,CA,60.457001,61.174999,247.408997,other
2,A,ARG,C,C,61.854000,61.002998,248.013000,other
3,A,ARG,O,O,62.028000,60.966999,249.235001,other
4,A,ARG,C,CB,59.650002,62.205002,248.222000,other
...,...,...,...,...,...,...,...,...
24301,I,NAG,O,O4,51.841000,72.031998,133.104004,other
24302,I,NAG,O,O5,49.580002,71.544998,135.994995,other
24303,I,NAG,O,O6,50.793999,74.183998,134.738998,other
24304,I,NAG,O,O7,46.188999,69.619003,133.772003,other


In [4]:
val_dataset.get(0)

1lbk
['A']
1 2


MyData(x=[1656, 21], pos=[1656, 3], num_nodes=1656, num_residues=208, chain=0         A
1         A
2         A
3         A
4         A
       ... 
1651    NaN
1652    NaN
1653    NaN
1654    NaN
1655    NaN
Name: chain, Length: 1656, dtype: object, edge_index=[2, 60788], edge_relations=[60788], edge_weights=[60788], num_relation=7, fold_functions=[2, 1], fold_masks=[2, 1195], chains=[1636], domains=[1636], domain_flag=[1656], lig_flag=[1656], prot_id='1lbk', type='fold')

In [ ]:
# test_dataset = CustomMultiTaskDataset(split='test', task='ec', gearnet=True, alpha_only=False, root_dir = './datasets/MultiTask')
train_dataset = CustomMultiTaskDataset(split='train_ec', task='ec', gearnet=True, alpha_only=False, root_dir = './datasets/MultiTask')

In [ ]:
pdb_annot_dict = gen_ec_labels()

In [ ]:
pdb_annot_dict

In [ ]:
root_dir = './output_info/'
json_files = ['enzyme_commission_uniprots.json', 'gene_ontology_uniprots.json', 'protein_protein_uniprots.json', 'protein_ligand_uniprots.json']
json_dirs = [os.path.join(root_dir, json_file) for json_file in json_files]

In [ ]:
go_uniprot_dict, go_info_dict = gen_protein_property_uniprots(json_dirs[1])

In [ ]:
go_labels, go_full_uniprot_dict = gen_go_labels(go_uniprot_dict)

In [ ]:
go_labels

In [ ]:
import torch
seed = int(torch.empty((), dtype=torch.int64).random_().item())
generator = torch.Generator()
generator.manual_seed(seed)

In [ ]:
rand_train = torch.randperm(24, generator=generator).tolist()

In [ ]:
rand_train

In [ ]:
# This script is for processing the dataset to form a new multi-task dataset with a unform labeling code
# Datasets PDBBind(Protein-Ligand, Protein-Protein), EnzymeCommission, GeneOntology
import os
import json
from typing import List
import random
import pandas as pd

def cal_complex_all(json_dirs):
    complex_list = []
    for json_dir in json_dirs:
        with open(json_dir, 'r') as f:
            info_dict = json.load(f)
        for i in info_dict:
            if '-' in i:
                id = i.split('-')[0].lower()
            complex_list.append(id)
    print(len(list(set(complex_list))))
        

def gen_train_test_ids(complex_dict, ec_dict, go_dict):
    train_list = []
    # print(complex_dict)
    all_list = []
    test_uniprots = []
    test_list_tmp = []
    for k, v in complex_dict.items():
        all_list.extend(v)
    all_list = list(set(all_list))
    for k, v in complex_dict.items():
        # print(k, v)
        # 由于每个蛋白可能有多个Uniprot，只能根据某一个Uniprot不在的情况找出训练集的样本
        if k not in ec_dict or k not in go_dict:
            train_list.extend(v)
    #取反即可获得测试集
    test_list = [i for i in all_list if i not in train_list]
    test_uniprots = []
    for k, v in complex_dict.items():
        for id in v:
            if id in test_list and k not in test_uniprots:
                test_uniprots.append(k)
    # print("Uniprots:", len(test_uniprots))
    # print("Train:", len(train_list), len(list(set(train_list))))
    # print("Test:", len(test_list), len(list(set(test_list))))
    # print("All:", len(list(set(test_list + train_list))))
    # print(list(set(train_list)))
    return list(set(train_list)), list(set(test_list)), list(set(test_uniprots))
        
            
def gen_protein_property_uniprots(json_dir: str, single=True):
    with open(json_dir, 'r') as f:
        info_dict = json.load(f)
    uniprot_dict = {} # 蛋白质 dictionary
    # print(len(info_dict))
    info_dict_new = {}
    for k, v in info_dict.items():
        if single:
            if len(v) != 1:
                continue
            else:
                uniprot_id = v[0]
                info_dict_new[k] = [uniprot_id]   
                # 只记录同一个uniprot id的所有pdb_id
                if uniprot_id not in uniprot_dict:     
                    uniprot_dict[uniprot_id] = k
                # else:
                #     uniprot_dict[uniprot_id].append(k)
        else:
            if len(v) == 0 or len(v) > 3:
                continue
            else:
                info_dict_new[k] = v
                for uniprot_id in v:
                    if uniprot_id not in uniprot_dict:
                        uniprot_dict[uniprot_id] = [k]
                    else:
                        uniprot_dict[uniprot_id].append(k)
    # print("Unitest:", len(uniprot_dict), len(info_dict_new))
    return uniprot_dict, info_dict_new
    
def save_dataset_info(dir, complex_list):
    with open(dir, 'w') as f:
        for complex in complex_list:
            f.write(complex + '\n')
        f.close()

def gen_ec_labels():
    root_dir = './data/EnzymeCommission/nrPDB-EC_annot.tsv'
    # root_dir = '/home/rhan21/sjm/GMSL/data/EnzymeCommission/nrPDB-EC_annot.tsv'
    with open(root_dir, 'r') as f:
        lines = f.readlines()
    ec_classes = lines[1].strip().split('\t')
    label_dict = {}
    pdb_annot_dict = {}
    label_id = 0
    for label in ec_classes:
        if label not in label_dict:
            label_dict[label] = label_id
            label_id += 1
    for item in lines[3:]:
        pdb_id, annotations = item.split('\t')
        annotations_list = annotations.strip().split(',')
        pdb_annot_dict[pdb_id] = [label_dict[annot] for annot in annotations_list]
    return pdb_annot_dict
    # print("Number of classes in task {} is {}".format('EnzymeCommission', label_id))

# def get_full_annotation(go_uniprot_dict):
#     root_dir = './data/GeneOntology/nrPDB-GO_annot.tsv'
#     with open(root_dir, 'r') as f:
#         lines = f.readlines()
#     go_classes_molecular_functions = lines[1].strip().split('\t')
#     go_classes_biological_process = lines[5].strip().split('\t')
#     go_classes_cellular_component = lines[9].strip().split('\t')
#     for k, v in go_uniprot_dict.items():
        

def gen_go_labels(go_uniprot_dict):
    root_dir = './data/GeneOntology/nrPDB-GO_annot.tsv'
    root_dir = '/home/rhan21/sjm/GMSL/data/GeneOntology/nrPDB-GO_annot.tsv'
    with open(root_dir, 'r') as f:
        lines = f.readlines()
    go_classes_molecular_functions = lines[1].strip().split('\t')
    go_classes_biological_process = lines[5].strip().split('\t')
    go_classes_cellular_component = lines[9].strip().split('\t')
    
    label_dict = {'molecular_functions':{}, 'biological_process':{}, 'cellular_component':{}}
    pdb_annot_dict = {}
    label_id_molecular = 0
    label_id_biological = 0
    label_id_cellular = 0
    full_ids = []
    full_uniprot_dict = {}
    for label in go_classes_molecular_functions:
        if label not in label_dict['molecular_functions']:
            label_dict['molecular_functions'][label] = label_id_molecular
            label_id_molecular += 1
    for label in go_classes_biological_process:
        if label not in label_dict['biological_process']:
            label_dict['biological_process'][label] = label_id_biological
            label_id_biological += 1
    for label in go_classes_cellular_component:
        if label not in label_dict['cellular_component']:
            label_dict['cellular_component'][label] = label_id_cellular
            label_id_cellular += 1
    for item in lines[13:]:
        pdb_id, molecular, biological, cellular = item.split('\t')
        molecular_list  = molecular.strip().split(',')
        biological_list = biological.strip().split(',')
        cellular_list = cellular.strip().split(',')
        # print(molecular_list)
        # 列表中会包含一些空的信息
        pdb_annot_dict[pdb_id] = {'molecular_functions':[label_dict['molecular_functions'][annot] for annot in molecular_list if annot != ''],
                                  'biological_process':[label_dict['biological_process'][annot] for annot in biological_list if annot != ''],
                                  'cellular_component':[label_dict['cellular_component'][annot] for annot in cellular_list if annot != '']}
        if pdb_annot_dict[pdb_id]['molecular_functions'] != [] and pdb_annot_dict[pdb_id]['biological_process'] != [] and pdb_annot_dict[pdb_id]['cellular_component'] != []:
            full_ids.append(pdb_id)
    for k, v in go_uniprot_dict.items():
        if v in full_ids:
            full_uniprot_dict[k] = v
    print("Number of classes in task {} is {}".format('molecular_functions', label_id_molecular+1))
    print("Number of classes in task {} is {}".format('biological_process', label_id_biological+1))
    print("Number of classes in task {} is {}".format('cellular_component', label_id_cellular+1))
    return pdb_annot_dict, full_uniprot_dict
    # print(pdb_annot_dict)


def gen_lba_labels():
    root_dir = './data/PDBbind/refined-set/index/INDEX_general_PL_data.2020'
    root_dir = '/home/rhan21/sjm/GMSL/data/PDBbind/refined-set/index/INDEX_general_PL_data.2020'
    res = {}
    with open(root_dir) as f:
        for line in f:
            if '#' in line:
                continue
            cont = line.strip().split()
            if len(cont) < 5:
                continue
            code, pk = cont[0], cont[3]
            res[code] = float(pk)
    print("LBA:", len(res))
    return res


def gen_ppi_labels():
    root_dir = './data/protein_protein/pp_affinity.xlsx'
    pp_info = pd.read_excel(root_dir, header=1)
    pdb_codes = pp_info['PDB code']
    res = {}
    for i, code in enumerate(pdb_codes):
        res[code] = pp_info['pKd pKi pIC50'][i]
    # print("PPI:", len(res))
    return res


def gen_label(pdb_ids, 
            ec_labels, go_labels, ppi_labels, lba_labels, 
            ec_uniprot_dict, go_uniprot_dict, 
            ec_info_dict, go_info_dict, pp_info_dict, pl_info_dict):
    uniform_labels = {}
    for pdb_id in pdb_ids:
        if '-' in pdb_id:
            if pdb_id in ec_labels:
                uniprots = ec_info_dict[pdb_id]
                single_ec_label = [ec_labels[pdb_id]]
            else:
                single_ec_label = [-1]
            if pdb_id in go_labels:
                uniprots = go_info_dict[pdb_id]
                single_go_label = [go_labels[pdb_id]]
            else:
                single_go_label = [-1]
        else:
            single_ec_label = []
            single_go_label = []
            if pdb_id in pp_info_dict:
                uniprots = pp_info_dict[pdb_id]
            elif pdb_id in pl_info_dict:
                uniprots = pl_info_dict[pdb_id]
            else:
                raise NotImplementedError
            for uniprot_id in uniprots:
                if uniprot_id in ec_uniprot_dict:
                    ec_label = ec_labels[ec_uniprot_dict[uniprot_id]]
                else:
                    ec_label = -1
                if uniprot_id in go_uniprot_dict:
                    go_label = go_labels[go_uniprot_dict[uniprot_id]]
                else:
                    go_label = -1
                single_ec_label.append(ec_label)
                single_go_label.append(go_label)
        if pdb_id in ppi_labels:
            ppi_label = ppi_labels[pdb_id]
        else:
            ppi_label = -1
        if pdb_id in lba_labels:
            lba_label = lba_labels[pdb_id]
        else:
            lba_label = -1
        uniform_labels[pdb_id] = {"uniprots":uniprots, "ec": single_ec_label, "go": single_go_label, "ppi": ppi_label, "lba": lba_label}
    return uniform_labels
    # print(uniform_labels)



In [ ]:
gen_lba_labels()

In [ ]:
def get_ec():
    root_dir = './output_info/'
    json_files = ['enzyme_commission_uniprots.json']
    json_dirs = [os.path.join(root_dir, json_file) for json_file in json_files]
    # cal_complex_all(json_dirs)
    ec_uniprot_dict, ec_info_dict = gen_protein_property_uniprots(json_dirs[0])
    # go_uniprot_dict, go_info_dict = gen_protein_property_uniprots(json_dirs[1])
    # pp_uniprot_dict, pp_info_dict = gen_protein_property_uniprots(json_dirs[2], single=False)
    # pl_uniprot_dict, pl_info_dict = gen_protein_property_uniprots(json_dirs[3], single=False)
    # print("pl, pp, ec, go:", len(pl_info_dict), len(pp_info_dict), len(ec_info_dict), len(go_info_dict))
    print("ec:",len(ec_info_dict))
    ec_labels = gen_ec_labels()

    
    full_test_list = test_list_pl[int(0.6*len(test_list_pl)):] + test_list_pp[int(0.6*len(test_list_pp)):]
    full_val_list = test_list_pl[int(0.2*len(test_list_pl)): int(0.6*(len(test_list_pl)))] + test_list_pp[int(0.2*len(test_list_pp)): int(0.6*len(test_list_pp))]
    full_train_list = test_list_pl[: int(0.2*len(test_list_pl))] + test_list_pp[: int(0.2*len(test_list_pp))]

    train_list_ec = [ec_uniprot_dict[i] for i in ec_uniprot_dict if i not in test_uniprots_all]
    train_list_go = [go_uniprot_dict[i] for i in go_uniprot_dict if i not in test_uniprots_all]
    print("Train List:", len(train_list_ec), len(train_list_go))
    train_list_all = list(set(train_list_pl + train_list_pp + train_list_ec + train_list_go + full_train_list))
    # print(train_list_all)
    print(len(full_test_list), len(full_val_list), len(train_list_all))
    print("save_info")
    save_dataset_info('./data/MultiTask/train_list_all.txt', train_list_all)
    save_dataset_info('./data/MultiTask/full_train.txt', full_train_list)
    save_dataset_info('./data/MultiTask/full_val.txt', full_val_list)
    save_dataset_info('./data/MultiTask/full_test.txt', full_test_list)
    
    print(len(ec_info_dict), len(ec_uniprot_dict), len(ec_labels))
    print(len(go_info_dict), len(go_uniprot_dict), len(go_labels))
    uniformed_label_dict = gen_label(train_list_all+full_test_list+full_val_list, ec_labels, go_labels, ppi_labels, lba_labels, ec_uniprot_dict, go_uniprot_dict, ec_info_dict, go_info_dict, pp_info_dict, pl_info_dict)    
    print(full_test_list[0], ": ", uniformed_label_dict[full_test_list[0]])
    with open('./data/MultiTask/uniformed_labels.json', 'w') as f:
        json.dump(uniformed_label_dict, f)

In [ ]:
save_dataset_info('/home/rhan21/sjm/GMSL/data/MultiTask/train_list_all.txt', train_list_all)
save_dataset_info('/home/rhan21/sjm/GMSL/data/MultiTask/full_train.txt', full_train_list)
save_dataset_info('/home/rhan21/sjm/GMSL/data/MultiTask/full_val.txt', full_val_list)
save_dataset_info('/home/rhan21/sjm/GMSL/data/MultiTask/full_test.txt', full_test_list)

In [ ]:
print(len(ec_info_dict), len(ec_uniprot_dict), len(ec_labels))
print(len(go_info_dict), len(go_uniprot_dict), len(go_labels))
uniformed_label_dict = gen_label(train_list_all+full_test_list+full_val_list, ec_labels, go_labels, ppi_labels, lba_labels, ec_uniprot_dict, go_uniprot_dict, ec_info_dict, go_info_dict, pp_info_dict, pl_info_dict)    
print(full_test_list[0], ": ", uniformed_label_dict[full_test_list[0]])
with open('./data/MultiTask/uniformed_labels.json', 'w') as f:
    json.dump(uniformed_label_dict, f)

In [ ]:
import os

In [ ]:
def get_args():
    # Multiprocess Setting to speedup dataloader
    torch.multiprocessing.set_start_method('forkserver')
    torch.set_float32_matmul_precision('high')
    args = get_argparse()

    

def go_task(args):
    device = args.device
    # if device != "cpu":
    #     if ',' not in device:
    #         device = [int(device)]
    #     else:
    #         device = [int(i) for i in device.split(',')]
    if args.wandb:
        name = args.run_name + time.strftime("%Y-%m-%d-%H-%M-%S")
        wandb.init(project='gmsl', name=name)
        wandb_logger = WandbLogger()
    else:
        wandb_logger = None
    model = PropertyModel(
            args=args,
            sdim=args.sdim,
            vdim=args.vdim,
            depth=args.depth,
            model_type=args.model_type,
            learning_rate=args.learning_rate,
            weight_decay=args.weight_decay,
            r_cutoff=4.5,
            num_radial=args.num_radial,
            max_epochs=args.max_epochs,
            factor_scheduler=0.75,
            enhanced=args.enhanced,
            offset_strategy = args.offset_strategy,
            task=args.train_task
        )
    model_cls = PropertyModel
    print(
            f"Model consists of {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable params."
        )
    model_dir = osp.join(MODEL_DIR, args.save_dir)
    if not osp.exists(model_dir):
        os.makedirs(model_dir)

    run_results = []
    seed = args.seed
    for run in range(args.nruns):
        pl.seed_everything(seed, workers=True)
        seed += run
        print(f"Starting run {run} with seed {seed}")
        datamodule = LBADataLightning(
            batch_size=args.batch_size,
            num_workers=args.num_workers,
            drop_last=args.drop_last,
            # sample_strategy = args.sample_strategy,
            train_task=args.train_task,
            train_split=args.train_split,
            val_split=args.val_split,
            test_split=args.test_split,
            gearnet=True if args.model_type=='gearnet' else False,
            alpha_only=args.alpha_only
            # auxiliary=None
        )
        model = model_cls(
            args=args,
            sdim=args.sdim,
            vdim=args.vdim,
            depth=args.depth,
            model_type=args.model_type,
            learning_rate=args.learning_rate,
            weight_decay=args.weight_decay,
            r_cutoff=4.5,
            num_radial=args.num_radial,
            max_epochs=args.max_epochs,
            factor_scheduler=0.75,
            aggr="mean",
            use_norm=False,
            enhanced=args.enhanced,
            offset_strategy = args.offset_strategy,
            task=args.train_task,
            # readout=args.readout
        )
        # 根据不同任务设置选择最优模型的方法
        monitor, mode = choose_monitor(args.train_task)
        
        checkpoint_callback = ModelCheckpoint(
            monitor=monitor,
            filename="model-{epoch:02d}-{val_eloss:.4f}",
            mode=mode,
            save_last=False,
            verbose=True,
        )

        trainer = pl.Trainer(
            devices=device if device != "cpu" else None,
            accelerator="gpu" if device != "cpu" else "cpu",
            max_epochs=args.max_epochs,
            precision=32,
            # amp_backend="native",
            callbacks=[
                checkpoint_callback,
                LearningRateMonitor(),
                ModelSummary(max_depth=2)
            ],
            default_root_dir=model_dir,
            gradient_clip_val=args.gradient_clip_val,
            accumulate_grad_batches=args.batch_accum_grad,
            logger=wandb_logger,
            strategy=DDPStrategy(find_unused_parameters=True)
        )

        start_time = datetime.now()

        trainer.fit(model, datamodule=datamodule, ckpt_path=args.load_ckpt)

        end_time = datetime.now()
        time_diff = end_time - start_time
        print(f"Training time: {time_diff}")

        # running test set
        _ = trainer.test(ckpt_path="best", datamodule=datamodule)
        res = model.res
        run_results.append(res)

    with open(os.path.join(model_dir, "res.json"), "w") as f:
        json.dump(run_results, f)

    # aggregate over runs..
    results_df = pd.DataFrame(run_results)
    print(results_df.describe())
    # for result in run_results:
    #     wandb_logger.log_metrics(result)